# Prática Integradora em Ciência de Dados para Negócios - Atividade 2

# Parte 1: Web Scraping

In [37]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from tqdm import tqdm
from datetime import datetime
import numpy as np

def configurar_headers():
    """Configura headers para evitar bloqueios"""
    return {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Accept-Language': 'en-US,en;q=0.9',
        'Accept-Encoding': 'gzip, deflate',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
        'Referer': 'https://www.google.com/'
    }

def extrair_dados_produto(produto):
    """Extrai dados de um único produto"""
    try:
        titulo = produto.h3.a['title']
        preco = produto.find('p', class_='price_color').get_text()
        disponibilidade = produto.find('p', class_='instock').get_text().strip()
        avaliacao = produto.find('p', class_='star-rating')['class'][1]
        
        return {
            'Título': titulo,
            'Preço': preco,
            'Disponibilidade': disponibilidade,
            'Avaliação': f"{avaliacao} estrelas"
        }
    except Exception as e:
        print(f"Erro ao extrair produto: {str(e)}")
        return None

def scrape_pagina(url, headers):
    """Faz scraping de uma página individual"""
    try:
        response = requests.get(url, headers=headers, timeout=15)
        response.encoding = 'utf-8'  # Força a codificação UTF-8
        response.raise_for_status()
        
        soup = BeautifulSoup(response.text, 'html.parser')
        produtos = soup.find_all('article', class_='product_pod')
        
        dados_pagina = []
        for produto in produtos:
            dados = extrair_dados_produto(produto)
            if dados:
                dados_pagina.append(dados)
        
        return dados_pagina
        
    except requests.exceptions.RequestException as e:
        print(f"Erro na requisição para {url}: {str(e)}")
        return []

def scrape_todas_paginas(base_url, max_paginas=3):
    """Faz scraping de múltiplas páginas"""
    headers = configurar_headers()
    todos_dados = []
    
    with tqdm(total=max_paginas, desc="Processando páginas") as pbar:
        for pagina in range(1, max_paginas + 1):
            if pagina == 1:
                url = base_url
            else:
                url = f"{base_url}/catalogue/page-{pagina}.html"
            
            dados_pagina = scrape_pagina(url, headers)
            if dados_pagina:
                todos_dados.extend(dados_pagina)
            pbar.update(1)
            time.sleep(1.5)  # Delay maior para evitar bloqueio
            
    return todos_dados

def salvar_dados(dados):
    """Salva os dados em CSV"""
    if not dados:
        print("Nenhum dado foi coletado. Verifique:")
        print("- Sua conexão com a internet")
        print("- Se o site está acessível (http://books.toscrape.com)")
        print("- Se os seletores CSS ainda são válidos")
        return
    
    df = pd.DataFrame(dados)
    df.to_csv('dados_livros.csv', index=False, encoding='utf-8-sig')
    print(f"\nDados salvos com sucesso em dados_livros.csv")
    print(f"Total de produtos coletados: {len(df)}")
    print("\nPrimeiros registros:")
    print(df.head())

def main():
    """Função principal"""
    print("Iniciando coleta de dados de livros...\n")
    base_url = "http://books.toscrape.com"
    
    # Teste de conexão com o site
    try:
        test_response = requests.get(base_url, timeout=10)
        test_response.raise_for_status()
    except:
        print("Erro: Não foi possível acessar o site. Verifique sua conexão com a internet.")
        return
    
    dados_livros = scrape_todas_paginas(base_url)
    salvar_dados(dados_livros)

if __name__ == "__main__":
    main()

Iniciando coleta de dados de livros...



Processando páginas: 100%|██████████| 3/3 [00:06<00:00,  2.19s/it]


Dados salvos com sucesso em dados_livros.csv
Total de produtos coletados: 60

Primeiros registros:
                                  Título   Preço Disponibilidade  \
0                   A Light in the Attic  £51.77        In stock   
1                     Tipping the Velvet  £53.74        In stock   
2                             Soumission  £50.10        In stock   
3                          Sharp Objects  £47.82        In stock   
4  Sapiens: A Brief History of Humankind  £54.23        In stock   

        Avaliação  
0  Three estrelas  
1    One estrelas  
2    One estrelas  
3   Four estrelas  
4   Five estrelas  


# Parte 2: Limpeza de Dados (simulando OpenRefine em Python)

In [38]:
def tratar_avaliacoes_corretamente(df):
    """Função corrigida para extrair avaliações em português/inglês"""
    print("\nIniciando tratamento de avaliações...")
    
    # Mostrar amostra dos dados brutos
    print("\nExemplo de avaliações brutas:")
    print(df['Avaliação'].head(10))
    
    # Criar dicionário de mapeamento para português e inglês
    rating_map = {
        'one': 1, 'um': 1, 'uma': 1,
        'two': 2, 'dois': 2, 'duas': 2,
        'three': 3, 'três': 3,
        'four': 4, 'quatro': 4,
        'five': 5, 'cinco': 5
    }
    
    # Processar cada avaliação
    avaliacoes_numericas = []
    for avaliacao in df['Avaliação']:
        # Converter para minúsculas e remover 'estrelas'
        avaliacao = str(avaliacao).lower().replace('estrelas', '').strip()
        
        # Procurar por correspondência no dicionário
        encontrou = False
        for termo, valor in rating_map.items():
            if termo in avaliacao:
                avaliacoes_numericas.append(valor)
                encontrou = True
                break
        
        if not encontrou:
            avaliacoes_numericas.append(0)  # Padrão para não avaliado
    
    df['Avaliação_Numérica'] = avaliacoes_numericas
    
    # Criar categorias
    conditions = [
        (df['Avaliação_Numérica'] == 0),
        (df['Avaliação_Numérica'] <= 2),
        (df['Avaliação_Numérica'] <= 4),
        (df['Avaliação_Numérica'] == 5)
    ]
    choices = ['Não avaliado', 'Ruim', 'Bom', 'Excelente']
    
    df['Avaliação_Categoria'] = np.select(conditions, choices, default='Não avaliado')
    
    print("\nDistribuição das avaliações corrigidas:")
    print(df['Avaliação_Categoria'].value_counts())
    
    return df

def tratamento_final_dados():
    """Função completa e corrigida de tratamento de dados"""
    try:
        # 1. Carregar dados
        df = pd.read_csv('dados_livros.csv', encoding='utf-8-sig')
        print(f"\nDados carregados. Total: {len(df)} registros")
        
        # 2. Remover duplicatas
        df.drop_duplicates(subset=['Título'], keep='first', inplace=True)
        print(f"Registros após remover duplicatas: {len(df)}")
        
        # 3. Tratar preços
        df['Preço'] = df['Preço'].str.replace('£', '').astype(float)
        df['Preço_BRL'] = df['Preço'] * 6.5
        
        # 4. Tratar avaliações (usando a função corrigida)
        df = tratar_avaliacoes_corretamente(df)
        
        # 5. Tratar disponibilidade
        df['Disponibilidade'] = df['Disponibilidade'].apply(
            lambda x: 'Disponível' if 'In stock' in str(x) else 'Indisponível'
        )
        
        # 6. Categorizar preços
        df['Categoria_Preço'] = pd.cut(
            df['Preço_BRL'],
            bins=[0, 50, 100, 200, float('inf')],
            labels=['Barato', 'Médio', 'Caro', 'Muito Caro']
        )
        
        # 7. Salvar dados tratados
        df.to_csv('dados_livros_tratados.csv', index=False, encoding='utf-8-sig')
        
        # Relatório final
        print("\n=== RESULTADOS FINAIS ===")
        print("\nDistribuição de avaliações:")
        print(df['Avaliação_Categoria'].value_counts())
        print("\nDistribuição de preços:")
        print(df['Categoria_Preço'].value_counts())
        print("\nAmostra dos dados finais:")
        print(df[['Título', 'Preço_BRL', 'Avaliação_Categoria', 'Disponibilidade']].head())
        
        return df
    
    except Exception as e:
        print(f"\nERRO: {str(e)}")
        return None

# Executar o tratamento completo
if __name__ == "__main__":
    dados_finais = tratamento_final_dados()


Dados carregados. Total: 60 registros
Registros após remover duplicatas: 60

Iniciando tratamento de avaliações...

Exemplo de avaliações brutas:
0    Three estrelas
1      One estrelas
2      One estrelas
3     Four estrelas
4     Five estrelas
5      One estrelas
6     Four estrelas
7    Three estrelas
8     Four estrelas
9      One estrelas
Name: Avaliação, dtype: object

Distribuição das avaliações corrigidas:
Avaliação_Categoria
Bom          23
Ruim         23
Excelente    14
Name: count, dtype: int64

=== RESULTADOS FINAIS ===

Distribuição de avaliações:
Avaliação_Categoria
Bom          23
Ruim         23
Excelente    14
Name: count, dtype: int64

Distribuição de preços:
Categoria_Preço
Muito Caro    35
Caro          22
Médio          3
Barato         0
Name: count, dtype: int64

Amostra dos dados finais:
                                  Título  Preço_BRL Avaliação_Categoria  \
0                   A Light in the Attic    336.505                 Bom   
1                     Tip

# Parte 3: Tratamento de Dados com Pandas


In [39]:
# --- Carregar os dados já limpos (saída da Parte 2) ---
df = pd.read_csv('dados_livros_tratados.csv', encoding='utf-8')

# --- 1. Verificação e ajuste final dos dados ---
print("\nVerificação inicial dos dados:")
print(df.info())

# Garantir tipos corretos
df['Preço'] = pd.to_numeric(df['Preço'], errors='coerce')
df['Avaliação_Numérica'] = pd.to_numeric(df['Avaliação_Numérica'], errors='coerce')

# --- 2. Filtragem de registros válidos ---
# Remover livros sem preço ou avaliação
df_filtrado = df.dropna(subset=['Preço', 'Avaliação_Numérica'])
df_filtrado = df_filtrado[df_filtrado['Preço'] > 0]

print(f"\nTotal de livros válidos para análise: {len(df_filtrado)}")

# --- 3. Cálculo de métricas (baseado nas categorias da Parte 2) ---
# Estatísticas básicas
estatisticas = {
    'Média de Preço (R$)': round(df_filtrado['Preço'].mean(), 2),
    'Mediana de Preço (R$)': round(df_filtrado['Preço'].median(), 2),
    'Média de Avaliação (1-5)': round(df_filtrado['Avaliação_Numérica'].mean(), 1),
    'Livros por Categoria de Preço': df_filtrado['Categoria_Preço'].value_counts().to_dict(),
    'Livros por Nível de Avaliação': df_filtrado['Avaliação_Categoria'].value_counts().to_dict()
}

# --- 4. Análise de Relação Preço x Avaliação ---
# Agrupar por categoria de preço e avaliação
analise_cruzada = pd.crosstab(
    df_filtrado['Categoria_Preço'],
    df_filtrado['Avaliação_Categoria'],
    margins=True
)

# --- 5. Salvar dados tratados para análise ---
df_filtrado.to_csv('dados_finais_para_analise.csv', index=False, encoding='utf-8-sig')

# --- 6. Resultados Finais ---
print("\n=== Estatísticas Finais ===")
for chave, valor in estatisticas.items():
    print(f"{chave}: {valor}")

print("\nDistribuição Cruzada (Preço x Avaliação):")
print(analise_cruzada)

print("\nAmostra dos Dados Finais:")
print(df_filtrado.sample(5))


Verificação inicial dos dados:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Título               60 non-null     object 
 1   Preço                60 non-null     float64
 2   Disponibilidade      60 non-null     object 
 3   Avaliação            60 non-null     object 
 4   Preço_BRL            60 non-null     float64
 5   Avaliação_Numérica   60 non-null     int64  
 6   Avaliação_Categoria  60 non-null     object 
 7   Categoria_Preço      60 non-null     object 
dtypes: float64(2), int64(1), object(5)
memory usage: 3.9+ KB
None

Total de livros válidos para análise: 60

=== Estatísticas Finais ===
Média de Preço (R$): 35.0
Mediana de Preço (R$): 33.48
Média de Avaliação (1-5): 3.0
Livros por Categoria de Preço: {'Muito Caro': 35, 'Caro': 22, 'Médio': 3}
Livros por Nível de Avaliação: {'Bom': 23, 'Ruim': 23, 'Excelente': 1

# Parte 4: Análise e Reflexão

In [ ]:
def gerar_relatorio_final_corrigido():
    """
    Gera relatório final com valores corrigidos conforme saída da Parte 3
    """
    # Dados reais obtidos na etapa 3
    dados = {
        'total_livros': 60,
        'livros_validos': 60,
        'preco_medio': 35.0,
        'preco_mediano': 33.48,
        'avaliacao_media': 3.0,
        'distribuicao_precos': {
            'Muito Caro': 35,
            'Caro': 22,
            'Médio': 3,
            'Barato': 0
        },
        'distribuicao_avaliacoes': {
            'Bom': 23,
            'Ruim': 23,
            'Excelente': 14
        },
        'relacao_preco_avaliacao': {
            'Caro': {'Bom': 7, 'Excelente': 7, 'Ruim': 8},
            'Muito Caro': {'Bom': 15, 'Excelente': 7, 'Ruim': 13},
            'Médio': {'Bom': 1, 'Excelente': 0, 'Ruim': 2}
        }
    }

    relatorio = f"""
# RELATÓRIO FINAL - ANÁLISE DE DADOS DE LIVROS

**Data:** {datetime.now().strftime('%d/%m/%Y %H:%M')}
**Fonte dos dados:** books.toscrape.com
**Total de livros analisados:** {dados['total_livros']}

## 1. METRADOS PRINCIPAIS

### Preços (em GBP):
- **Média:** £{dados['preco_medio']:.2f}
- **Mediana:** £{dados['preco_mediano']:.2f}

### Avaliações (1-5 estrelas):
- **Média:** {dados['avaliacao_media']:.1f}/5.0

## 2. DISTRIBUIÇÕES

### Categorias de Preço:
{formatar_distribuicao(dados['distribuicao_precos'])}

### Níveis de Avaliação:
{formatar_distribuicao(dados['distribuicao_avaliacoes'])}

## 3. RELAÇÃO PREÇO × AVALIAÇÃO

{formatar_relacao_preco_avaliacao(dados['relacao_preco_avaliacao'])}

## 4. OBSERVAÇÕES IMPORTANTES

1. **Distribuição de preços:** 
   - 58% dos livros estão na categoria "Muito Caro"
   - Apenas 5% na categoria "Médio"

2. **Avaliações equilibradas:**
   - Distribuição quase igual entre "Bom" e "Ruim"
   - 23% com avaliação "Excelente"

3. **Relação preço-avaliação:**
   - Livros "Muito Caros" têm maior variação de avaliações
   - Categoria "Caro" apresenta melhor equilíbrio qualidade-preço
"""

    # Salvar relatório
    with open('relatorio_final_corrigido.txt', 'w', encoding='utf-8') as f:
        f.write(relatorio)

    print("Relatório corrigido gerado com sucesso!")

# Funções auxiliares atualizadas
def formatar_distribuicao(dados):
    """Formata dados de distribuição com porcentagens"""
    total = sum(dados.values())
    return "\n".join(
        f"- {k}: {v} livros ({v/total:.1%})"
        for k, v in dados.items()
    )

def formatar_relacao_preco_avaliacao(dados):
    """Formata a tabela de relação preço×avaliação"""
    tabela = []
    for categoria, avaliacoes in dados.items():
        total = sum(avaliacoes.values())
        linha = [f"**{categoria}**"]
        linha.extend(f"{k}: {v} ({v/total:.1%})" for k, v in avaliacoes.items())
        tabela.append(" | ".join(linha))
    
    header = "| Categoria | Bom | Excelente | Ruim |\n|-----------|-----|-----------|------|"
    return header + "\n" + "\n".join(f"| {linha} |" for linha in tabela)

# Executar
if __name__ == "__main__":
    gerar_relatorio_final_corrigido()

Relatório corrigido gerado com sucesso!
